# Memoris project

## Imports

In [ ]:
from striplog import Component, Decor, Legend, Lexicon, Striplog
from core.visual import Borehole3D, striplog_legend_to_omf_legend 
from utils.lexicon_memoris import LEG_CONTAMINATION_LEV
from utils.config import DEFAULT_LITHO_LEXICON, DEFAULT_LITHO_LEGEND, DEFAULT_POL_LEXICON
from utils.io import dataframe_viewer, dict_viewer
from utils.orm import orm_boreholes_from_dataframe
from utils.visual import legend_from_attributes
from core.project import Project
from core.orm import BoreholeOrm, PositionOrm, Base 

import matplotlib.pyplot as plt
import os
import re
import pandas as pd
import numpy as np
from sqlalchemy import create_engine
from sqlalchemy.orm import sessionmaker
from definitions import ROOT_DIR

## Reading of CSV files in a dir

In [ ]:
data_dir = ROOT_DIR + '/CF_data/Result_traitem/fusion_finale/filtered/' 

In [ ]:
litho_file = data_dir + 'Lithologies.csv'
sample_file = data_dir + 'Samples.csv'

In [ ]:
litho = pd.read_csv(litho_file, sep=',')
sample = pd.read_csv(sample_file, sep=',')

In [ ]:
dataframe_viewer(litho, rows=5, un_val='ID'), dataframe_viewer(sample, rows=5, un_val='ID')

In [ ]:
litho.loc[1573, 'Litho_base'] = 4.5

In [ ]:
data_list = [litho, sample]
bh_id='F81'
cols=['Long_for', '_top', '_base']

for data in data_list:
    view_cols = []
    for col in data.columns:
        for c in cols:
            if re.search(c, col, re.I):
                view_cols.append(col)
                
    dataframe_viewer(data.query(f"ID=='{bh_id}'")[view_cols], rows=3, cols=16)

## Create pollutants list from the dataframe columns name

In [ ]:
pollutants = []
skipped = []
for i, c in enumerate(sample.columns):
    if c in DEFAULT_POL_LEXICON.abbreviations.keys() or c in DEFAULT_POL_LEXICON.abbreviations.values():
        #print(f"{i}: {c}")
        pollutants.append(c)
    else:
        skipped.append(c)
print(pollutants)

## Create Boreholes and components from dataframes

In [ ]:
data_list = [litho[:500], sample[:500]]

In [ ]:
boreholes_orm, components, link_intv_comp = orm_boreholes_from_dataframe(
                                                data_list, default_z=102, id_col='ID',
                                                length_col = 'Long_for', diameter_col='Diam_for',
                                                sample_type_col='Type_ech', sample_id_col='ID_ech',
                                                attributes=['lithology']+pollutants, 
                                                symbols={'lithology':{'lexicon':DEFAULT_LITHO_LEXICON}}
                                                )

In [ ]:
len(boreholes_orm), len(components)

In [ ]:
boreholes_orm # list of core.BoreholeOrm objects

In [ ]:
components # dict of striplog.Component objects

In [ ]:
boreholes_orm[0].intervals

In [ ]:
i = 0
components[i][list(components[i].keys())[0]]#.json()

## Create a dictionary containing legends for each representation attribute

In [ ]:
legend_dict = legend_from_attributes([('lithology', DEFAULT_LITHO_LEGEND)]+pollutants)

## Create a database for the project

In [ ]:
DB_name = '../tmp_files/memoris_db.db'

if os.path.exists(DB_name):
    os.remove(DB_name)

engine = create_engine(f"sqlite:///{DB_name}", echo=False)

In [ ]:
Base.metadata.create_all(engine)

In [ ]:
Session = sessionmaker(bind=engine)
session = Session()

## create a project

In [ ]:
p = Project(session, name='Memoris_project', legend_dict=legend_dict, lexicon=DEFAULT_LITHO_LEXICON)

### Insert boreholes and components into the project

In [ ]:
p.add_components(components)

In [ ]:
for bhorm in boreholes_orm:
    p.add_borehole(bhorm)

In [ ]:
p.add_link_components_intervals(link_intv_comp)

### Validate modifications 

In [ ]:
p.commit()

In [ ]:
boreholes_orm[0].intervals[0].components

In [ ]:
p.refresh()

In [ ]:
session.close()

## update project legend dict

In [ ]:
p.update_legend_cmap(compute_all_attrib=True, legend_dict=legend_dict, verbose=False)

In [ ]:
p.legend_dict#['lithology']['legend']

## Visualization

In [ ]:
attributes = list(legend_dict.keys())
attributes

In [ ]:
n = 0
bh = 1
repr_att = attributes[n]

### Log plot for one borehole

In [ ]:
p.boreholes_3d[0].intervals

In [ ]:
p.boreholes_3d[n].attrib_components(attribute=repr_att)

In [ ]:
p.boreholes_3d[n].components 
# lacking properties in components, why ???

In [ ]:
p.boreholes_3d[n].plot_log(repr_attribute=repr_att)

### 3D display of one borehole

In [ ]:
at = 1
n = 5
repr_att = attributes[at]
list(p.boreholes_3d[n].legend_dict.keys())[at]

In [ ]:
p.legend_dict[repr_att]['cmap']

In [ ]:
p.boreholes_3d[n].geometry.data[at].array.array

In [ ]:
p.boreholes_3d[n].plot_3d(repr_attribute=repr_att, str_annotations=False)

In [ ]:
p.boreholes_3d[n]._vtk.CELL_DATA_FIELD #.get_array #geometry.data[2].array.array

In [ ]:
from core.visual import Borehole3D
from striplog import Interval, Component

In [ ]:
intv = [Interval(0,1, components=[Component({'lithology':'sable'})]), 
        Interval(1,2, components=[Component({'lithology':'limon'})]),
        Interval(2,3, components=[Component({'lithology':'argile'})]),
        Interval(3,4, components=[Component({'lithology':'inconnu'})])]

In [ ]:
bh_test = Borehole3D({'lithology':intv, 'sample':None}, repr_attribute='lithology', length=4, legend_dict=legend_dict)

In [ ]:
bh_test.plot_log()#3d()

In [ ]:
p.boreholes_3d[n]._vtk

### Localization of project boreholes

### 3D display of project boreholes

In [ ]:
pause

In [ ]:
p.plot_3d(repr_attribute=repr_att, labels_size=15, bg_color=["royalblue", "aliceblue"], 
         window_size=(800, 600), verbose=True)

## Testing zone